Import PySpark modules

In [2]:
from pyspark import SparkConf
from pyspark import SparkContext

Import the NLTK package

In [15]:
import nltk

Create a SparkContext that uses the YARN resource manager and specifies a name for the application

In [4]:
conf = SparkConf()
conf.setMaster('yarn-client')
conf.setAppName('spark-nltk')
sc = SparkContext(conf=conf)

### RDD Creation

Specify the location of the data on the cluster nodes

In [52]:
data = sc.wholeTextFiles('file:///usr/share/nltk_data/corpora/state_union/')

In [53]:
# data = sc.textFile('file:///usr/share/nltk_data/corpora/state_union/')

In [54]:
data.take(1)

[(u'file:/usr/share/nltk_data/corpora/state_union/1961-Kennedy.txt',
  u'PRESIDENT JOHN F. KENNEDY\'S SPECIAL MESSAGE TO THE CONGRESS ON URGENT NATIONAL NEEDS\n \nMay 25, 1961\n\nMr. Speaker, Mr. Vice President, my copartners in Government, gentlemen-and ladies:\nThe Constitution imposes upon me the obligation to "from time to time give to the Congress information of the State of the Union." While this has traditionally been interpreted as an annual affair, this tradition has been broken in extraordinary times.\nThese are extraordinary times. And we face an extraordinary challenge. Our strength as well as our convictions have imposed upon this nation the role of leader in freedom\'s cause.\nNo role in history could be more difficult or more important. We stand for freedom.\nThat is our conviction for ourselves--that is our only commitment to others. No friend, no neutral and no adversary should think otherwise. We are not against any man--or any nation--or any system--except as it is h

### Word count

In [51]:
year = data.map(lambda x: x[0].split('-')[0].split('state_union/')[1])
year.collect()

[u'1961',
 u'2001',
 u'1992',
 u'1958',
 u'1994',
 u'1963',
 u'1996',
 u'2002',
 u'1991',
 u'1957',
 u'1988',
 u'1980',
 u'1997',
 u'1995',
 u'1956',
 u'1966',
 u'1970',
 u'2005',
 u'1977',
 u'1985',
 u'1953',
 u'1950',
 u'1998',
 u'1969',
 u'1946',
 u'1968',
 u'2006',
 u'1960',
 u'1976',
 u'1974',
 u'1962',
 u'1955',
 u'1999',
 u'1949',
 u'1979',
 u'1971',
 u'1959',
 u'1972',
 u'1990',
 u'1973',
 u'1945',
 u'1965',
 u'1947',
 u'1981',
 u'1989',
 u'1964',
 u'1978',
 u'README',
 u'1954',
 u'1984',
 u'2004',
 u'1967',
 u'1951',
 u'1975',
 u'1965',
 u'1948',
 u'1963',
 u'2001',
 u'2000',
 u'2003',
 u'1993',
 u'1986',
 u'1987',
 u'1983',
 u'1982',
 u'1991']

Map the text data to the word_tokenize function in NLTK

In [38]:
words = data.map(lambda x: len(nltk.word_tokenize(x[1])))

Return ten of the words in the dataset

In [44]:
print words.collect()

[6550, 3578, 5896, 5510, 8432, 1857, 7130, 4701, 4555, 4587, 5577, 3808, 7640, 10418, 9056, 6199, 4987, 6018, 5286, 4836, 7714, 5611, 8383, 4617, 30004, 5600, 6462, 6230, 5534, 5757, 7485, 8089, 8586, 3769, 3715, 4511, 5552, 4472, 4415, 1860, 2108, 4189, 6623, 5043, 5582, 3626, 5172, 30, 6653, 5684, 6213, 8150, 4450, 4668, 4922, 5614, 6133, 5369, 10387, 6139, 7810, 4012, 4392, 6310, 5911, 3292]


NTLK’s POS-tagger can be used to find the part of speech for each word

In [8]:
pos_word = words.map(lambda x: nltk.pos_tag([x]))

Return five of the words in the dataset

In [9]:
print pos_word.take(5)

[[(u'Address', 'NN')], [(u'on', 'IN')], [(u'the', 'DT')], [(u'State', 'NN')], [(u'of', 'IN')]]


### Named Entity Recognition

Import named-entity recognition library.

In [65]:
import geograpy

In [56]:
places = data.map(lambda x: x[0] + "\t\t" + str(len(x[1])))

In [57]:
places.take(5)

[u'file:/usr/share/nltk_data/corpora/state_union/1961-Kennedy.txt\t\t35090',
 u'file:/usr/share/nltk_data/corpora/state_union/2001-GWBush-2.txt\t\t18036',
 u'file:/usr/share/nltk_data/corpora/state_union/1992-Bush.txt\t\t28674',
 u'file:/usr/share/nltk_data/corpora/state_union/1958-Eisenhower.txt\t\t30324',
 u'file:/usr/share/nltk_data/corpora/state_union/1994-Clinton.txt\t\t42133']

In [73]:
text = "My name is Zach Carwile and I live in the United States."
test = geograpy.get_place_context(text=unicode(text,encoding='utf-8', errors='ignore'))
print(test)

In [83]:
def getPlaces(text):
    places = geograpy.get_place_context(text=text) # WHAT'S WRONG WITH THIS ONE LINE??!??
    return places

In [84]:
places = data.map(lambda x: getPlaces(x[1]))
places.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 37.0 failed 4 times, most recent failure: Lost task 0.3 in stage 37.0 (TID 96, ip-172-31-4-74.ec2.internal): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/anaconda/share/spark/python/pyspark/worker.py", line 98, in main
    command = pickleSer._read_with_length(infile)
  File "/opt/anaconda/share/spark/python/pyspark/serializers.py", line 164, in _read_with_length
    return self.loads(obj)
  File "/opt/anaconda/share/spark/python/pyspark/serializers.py", line 422, in loads
    return pickle.loads(obj)
  File "/opt/anaconda/share/spark/python/pyspark/cloudpickle.py", line 653, in subimport
    __import__(name)
  File "/opt/anaconda/lib/python2.7/site-packages/geograpy/__init__.py", line 2, in <module>
    from places import PlaceContext
  File "/opt/anaconda/lib/

### Visualization